In [ ]:
!pip install  hazm

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 892.6/892.6 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 47.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.3/243.3 kB 22.4 MB/s eta 0:00:00
  Created wheel for flashtext: filename=flashtext-2.7-py2.py3-none-any.whl size=9298 sha256=561def54810b7cdea4148d3efe93967ac95c65f6ad190f888f750c7a0f9ac35a
  Stored in directory: /root/.cache/pip/wheels/49/20/47/f03dfa8a7239c54cbc44ff7389eefbf888d2c1873edaaec888
Successfully built flashtext
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviou

In [ ]:
import nltk
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords as nltk_stopwords
from nltk.tag import pos_tag
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
import hazm

In [ ]:
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('stopwords')

stop_words = set(nltk_stopwords.words('english') + [
    'when', 'now', 'then', 'soon', 'today', 'tomorrow', 'yesterday', 'always', 'never', 'later',
    'early', 'after', 'before', 'during', 'while', 'often', 'recently', 'soonest', 'time', 'also',
    'personal', 'us', 'is', 'am', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had',
    'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because',
    'in', 'on', 'at', 'with', 'as', 'by', 'of', 'to', 'for', 'from', 'about', 'that', 'which', 'who',
    'whom', 'this', 'these', 'those', 'i', 'you', 'he', 'she', 'it', 'we', 'they'
] + [
    'و', 'در', 'به', 'از', 'که', 'این', 'با', 'برای', 'اینکه', 'بود', 'کند', 'وقتی', 'حالا', 'است',
    'دهی', 'بود', 'با', 'برای', 'اینکه', 'که', 'در', 'به', 'از', 'روی', 'تا', 'و', 'یا', 'این',
    'آن', 'آنها', 'من', 'تو', 'او', 'ما', 'شما', 'ایشان', 'که', 'بودند', 'های', 'دارند', 'باشد',
    'می', 'هم', 'اینطور', 'اینگونه', 'همچنین', 'پس', 'چرا', 'چگونه', 'کجا', 'باید', 'هر', 'یک',
    'مورد', 'چیزی', 'همه', 'هیچ', 'فقط', 'نه', 'همه', 'کرده', 'سپس', 'زود', 'امروز', 'فردا','بلکه','بسیار', 'باشید', 'آماده'
    'دیروز', 'همیشه', 'هرگز', 'بعد', 'زودتر', 'بعد از', 'قبل از', 'در طول', 'در حین', 'اغلب','دنیای',
    'اخیراً', 'خیلی زود', 'زمان', 'ها', 'نه', 'تا', 'کسانی', 'آن', 'یک', 'هم', 'اینها', 'او',
    'آنها', 'ما', 'شما', 'مهم', 'که', 'می', 'شود', 'همه', 'بیشتر', 'اول', 'هایی', 'های', 'همچنین',
    'نیز', 'باید', 'مجموعه', 'حالا', 'کرد', 'نشان', 'میکند', 'درست', 'دیگر', 'چند', 'هیچ', 'کمی',
    'جدید', 'چطور', 'توسط', 'اگر', 'بنابراین', 'هنگام', 'پس', 'اما', 'بیشتر', 'درباره', 'غیر',
    'برای', 'که', 'چند', 'همه', 'وضعیت', 'نظری', 'چنین', 'خواهید', 'نکته', 'آیا', 'باشد', 'بر',
    'درون', 'یکی', 'طی', 'یا', 'طور', 'مبتنی', 'کار', 'شد', 'ساخت', 'خود', 'مورد', 'درخواست',
    'نکات', 'آنان', 'است', 'دهد', 'کامل', 'باش', 'شخصی', 'صرف', 'زیاد', 'مقابل', 'هنوز', 'تعداد',
    'میزان', 'حدود', 'مگر', 'از آن', 'به طور', 'در اینجا', 'بیشتر از', 'کمتر از', 'همچنین',
    'برای اینکه', 'حتی', 'طی زمان', 'دقیقا', 'اینطور', 'اینچنین', 'دارند', 'تواند', 'را', 'کن',
    'کرد', 'کردن', 'باش', 'بود', 'بودن', 'شو', 'شد', 'شدن', 'داشت', 'داشتن', 'خواه', 'خواست',
    'خواستن', 'گوی', 'گفت', 'گفتن', 'گیر', 'گرفت', 'گرفتن', 'آی', 'آمد', 'آمدن', 'توان', 'توانست',
    'توانستن', 'یاب', 'یافت', 'یافتن', 'آور', 'آورد', 'آوردن','اخیرا', 'اینجا', 'حین', 'خیلی', 'طول', 'قبل', 'کمتر'
])




[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
normalizer = hazm.Normalizer()
lemmatizer = hazm.Lemmatizer()

def preprocess_persian(text):
    text = normalizer.normalize(text)
    tokens = hazm.word_tokenize(text)
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stop_words]
    return ' '.join(tokens)


In [ ]:
def extract_topics_and_keywords(text, num_keywords=8, num_topics=4, is_persian=False):

    if is_persian:
        processed_text = preprocess_persian(text)
        texts = [processed_text]
    else:
        texts = [text.lower()]

    vectorizer = TfidfVectorizer(stop_words=list(stop_words), max_features=300)
    X = vectorizer.fit_transform(texts)
    keywords = vectorizer.get_feature_names_out()[:num_keywords]

    tokens = word_tokenize(text.lower())
    filtered_tokens = [word for word in tokens if word.isalnum() and word not in stop_words]
    tagged_tokens = pos_tag(filtered_tokens)
    keyword_candidates = [word for word, pos in tagged_tokens if pos.startswith('NN') or pos.startswith('JJ')]
    keyword_freq = Counter(keyword_candidates)
    topic_candidates = keyword_freq.most_common(15)

    theme_keywords = {
        '(فناوری/technology)': [
            # کلمات انگلیسی
            'technology', 'smartphones', 'intelligence', 'artificial', 'innovations', 'software', 'hardware',
            'devices', 'digital', 'internet', 'computing', 'programming', 'data', 'automation', 'robotics',
            'gadgets', 'electronics', 'networks', 'systems', 'tech', 'apps', 'cloud', 'cybersecurity',
            'algorithms', 'virtual', 'augmented', 'reality', 'machine', 'learning', 'blockchain', 'quantum',
            'iot', 'interface', 'connectivity', 'wireless', 'mobile', 'telecommunication', 'platform',
            'interface', 'encryption', 'servers', 'databases', 'processors', 'circuits', 'signals', 'bandwidth',
            'streaming', 'wearables', 'sensors', 'analytics', 'development', 'coding', 'debugging', 'updates',
            'integration', 'compatibility', 'standards', 'protocols', 'infrastructure', 'satellites', 'nano',
            'biotech', 'grids', 'smart', 'optimization', 'simulation', 'rendering', 'hardware acceleration',
            # کلمات فارسی
            'فناوری', 'گوشی‌های هوشمند','هوش مصنوعی' ,'هوش', 'مصنوعی', 'نوآوری', 'نرم‌افزار', 'سخت‌افزار', 'دستگاه',
            'دیجیتال', 'اینترنت', 'رایانش', 'برنامه‌نویسی', 'داده', 'اتوماسیون', 'رباتیک', 'ابزارک',
            'الکترونیک', 'شبکه', 'سیستم', 'تکنولوژی', 'اپلیکیشن', 'ابر', 'امنیت سایبری', 'الگوریتم',
            'مجازی', 'واقعیت افزوده', 'ماشین', 'یادگیری', 'بلاک‌چین', 'کوانتومی', 'اینترنت اشیا',
            'رابط', 'اتصال', 'بی‌سیم', 'موبایل', 'ارتباطات', 'هوشمندسازی', 'پردازش', 'فناوری اطلاعات',
            'پلتفرم', 'رمزنگاری', 'سرور', 'پایگاه داده', 'پردازنده', 'مدار', 'سیگنال', 'پهنای باند',
            'استریمینگ', 'گجت‌های پوشیدنی', 'حسگر', 'تحلیل داده', 'توسعه', 'کدنویسی', 'اشکال‌زدایی',
            'به‌روزرسانی', 'یکپارچه‌سازی', 'سازگاری', 'استاندارد', 'پروتکل', 'زیرساخت', 'ماهواره',
            'نانو', 'بیوتکنولوژی', 'شبکه هوشمند', 'بهینه‌سازی', 'شبیه‌سازی', 'رندرینگ', 'شتاب‌دهنده سخت‌افزاری'
        ],
        '(فرهنگ/culture)': [
            # کلمات انگلیسی
            'cultural', 'traditions', 'cultures', 'unity', 'heritage', 'customs', 'identity', 'history',
            'diversity', 'values', 'beliefs', 'celebrations', 'rituals', 'ethnicity', 'community', 'language',
            'art', 'music', 'literature', 'festivals', 'folklore', 'religion', 'mythology', 'symbols', 'dance',
            'crafts', 'cuisine', 'architecture', 'poetry', 'theater', 'tradition', 'legacy', 'folktales',
            'spirituality', 'philosophy', 'aesthetics', 'ethics', 'norms', 'practices', 'ceremonies', 'dress',
            'fashion', 'songs', 'stories', 'paintings', 'sculpture', 'cinema', 'media', 'heritage sites',
            'folksongs', 'oral history', 'languages', 'dialects', 'proverbs', 'social customs', 'belief systems',
            # کلمات فارسی
            'فرهنگی', 'سنت', 'فرهنگ‌ها', 'وحدت', 'میراث', 'رسوم', 'هویت', 'تاریخ', 'تنوع', 'ارزش‌ها',
            'باورها', 'جشن‌ها', 'آیین', 'قومیت', 'جامعه', 'زبان', 'هنر', 'موسیقی', 'ادبیات', 'جشنواره',
            'فولکلور', 'دین', 'اساطیر', 'نماد', 'رقص', 'صنایع دستی', 'آشپزی', 'معماری', 'شعر', 'تئاتر',
            'سنت‌ها', 'میراث فرهنگی', 'هویت ملی', 'مراسم', 'آداب', 'داستان‌های عامیانه', 'معنویت',
            'فلسفه', 'زیبایی‌شناسی', 'اخلاق', 'هنجار', 'عملکرد', 'لباس', 'مد', 'آهنگ', 'داستان',
            'نقاشی', 'مجسمه‌سازی', 'سینما', 'رسانه', 'مکان‌های میراثی', 'آوازهای محلی', 'تاریخ شفاهی',
            'زبان‌ها', 'گویش', 'ضرب‌المثل', 'رسوم اجتماعی', 'نظام باورها'
        ],
        '(سلامتی/health)': [
            # کلمات انگلیسی
            'healthy', 'lifestyle', 'exercise', 'mindfulness', 'well-being', 'fitness', 'nutrition', 'diet',
            'mental', 'physical', 'wellness', 'stress', 'relaxation', 'medicine', 'disease', 'therapy',
            'energy', 'balance', 'sleep', 'recovery', 'healthcare', 'prevention', 'hygiene', 'strength',
            'endurance', 'yoga', 'meditation', 'immune', 'vitality', 'rehabilitation', 'sports', 'activity',
            'cardio', 'muscle', 'bones', 'posture', 'breathing', 'hydration', 'vitamins', 'supplements',
            'checkups', 'diagnosis', 'treatment', 'surgery', 'pharmacy', 'hospitals', 'doctors', 'nurses',
            'patients', 'mental health', 'anxiety', 'depression', 'therapy sessions', 'fitness goals',
            'weight', 'training', 'stamina', 'longevity', 'disability',
            # کلمات فارسی
            'سالم', 'سبک زندگی', 'ورزش', 'ذهن‌آگاهی', 'رفاه', 'تناسب اندام', 'تغذیه', 'رژیم', 'ذهنی',
            'جسمی', 'تندرستی', 'استرس', 'آرامش', 'پزشکی', 'بیماری', 'درمان', 'انرژی', 'تعادل', 'خواب',
            'بهبودی', 'بهداشت', 'پیشگیری', 'سلامت', 'قدرت', 'استقامت', 'یوگا', 'مدیتیشن', 'ایمنی',
            'سرزندگی', 'بازسازی', 'ورزش‌ها', 'فعالیت', 'تن‌درستی', 'روان', 'کاردیو', 'عضله', 'استخوان',
            'وضعیت بدنی', 'تنفس', 'آبرسانی', 'ویتامین', 'مکمل', 'معاینه', 'تشخیص', 'جراحی', 'داروخانه',
            'بیمارستان', 'پزشک', 'پرستار', 'بیمار', 'سلامت روان', 'اضطراب', 'افسردگی', 'جلسات درمانی',
            'اهداف تناسب اندام', 'وزن', 'تمرین', 'استقامت بدنی', 'طول عمر', 'ناتوانی'
        ],
        '(زندگی/life)': [
            # کلمات انگلیسی
            'lives', 'daily', 'personal', 'society', 'existence', 'routine', 'family', 'relationships', 'home',
            'social', 'people', 'individual', 'community', 'living', 'happiness', 'challenges', 'growth',
            'experience', 'modern', 'world', 'lifestyle', 'friends', 'love', 'peace', 'freedom', 'dreams',
            'journey', 'purpose', 'habits', 'adventure', 'connection', 'identity', 'belonging', 'emotions',
            'memories', 'values', 'goals', 'time', 'aging', 'youth', 'childhood', 'adulthood', 'parenting',
            'marriage', 'divorce', 'culture', 'leisure', 'hobbies', 'travel', 'nature', 'spirituality',
            'fulfillment', 'struggle', 'resilience', 'death', 'birth', 'legacy',
            # کلمات فارسی
            'زندگی', 'روزمره', 'شخصی', 'جامعه', 'هستی', 'روال', 'خانواده', 'روابط', 'خانه', 'اجتماعی',
            'مردم', 'فرد', 'اجتماع', 'زنده', 'خوشحالی', 'چالش', 'رشد', 'تجربه', 'مدرن', 'جهان', 'روش زندگی',
            'دوستان', 'عشق', 'آرامش', 'آزادی', 'رویاها', 'سفر', 'هدف', 'عادت', 'ماجرا', 'ارتباط', 'هویت',
            'تعلق', 'روزگار', 'معنا', 'احساسات', 'خاطرات', 'ارزش‌ها', 'اهداف', 'زمان', 'پیری', 'جوانی',
            'کودکی', 'بزرگسالی', 'فرزندپروری', 'ازدواج', 'طلاق', 'فرهنگ', 'اوقات فراغت', 'سرگرمی',
            'سفر کردن', 'طبیعت', 'معنویت', 'رضایت', 'مبارزه', 'مقاومت', 'مرگ', 'تولد', 'میراث'
        ],
        '(کار/work)': [
            # کلمات انگلیسی
            'work', 'communication', 'job', 'career', 'employment', 'profession', 'business', 'tasks',
            'productivity', 'skills', 'office', 'team', 'collaboration', 'management', 'projects', 'industry',
            'labor', 'effort', 'success', 'performance', 'workplace', 'leadership', 'strategy', 'operations',
            'innovation', 'responsibility', 'goals', 'enterprise', 'trade', 'occupation', 'duty', 'efficiency',
            'meetings', 'deadlines', 'salary', 'promotion', 'training', 'colleagues', 'clients', 'services',
            'production', 'sales', 'marketing', 'logistics', 'resources', 'planning', 'execution', 'remote',
            'freelance', 'consulting', 'negotiation', 'contracts', 'benefits', 'retirement', 'work-life balance',
            # کلمات فارسی
            'کار', 'ارتباط', 'شغل', 'حرفه', 'اشتغال', 'کسب‌وکار', 'وظایف', 'بهره‌وری', 'مهارت', 'دفتر',
            'تیم', 'همکاری', 'مدیریت', 'پروژه', 'صنعت', 'کارگری', 'تلاش', 'موفقیت', 'عملکرد', 'محل کار',
            'رهبری', 'استراتژی', 'عملیات', 'نوآوری', 'مسئولیت', 'اهداف', 'شرکت', 'تجارت', 'وظیفه',
            'کارایی', 'فعالیت حرفه‌ای', 'پیشرفت شغلی', 'جلسات', 'مهلت', 'حقوق', 'ترفیع', 'آموزش',
            'همکاران', 'مشتریان', 'خدمات', 'تولید', 'فروش', 'بازاریابی', 'لجستیک', 'منابع', 'برنامه‌ریزی',
            'اجرا', 'دورکاری', 'آزادکاری', 'مشاوره', 'مذاکره', 'قرارداد', 'مزایا', 'بازنشستگی',
            'تعادل کار و زندگی'
        ],
        '(آموزش/education)': [
            # کلمات انگلیسی
            'education', 'learning', 'knowledge', 'study', 'school', 'students', 'teachers', 'training',
            'skills', 'development', 'research', 'information', 'understanding', 'academics', 'classes',
            'lessons', 'scholarship', 'intellect', 'wisdom', 'teaching', 'curriculum', 'university', 'science',
            'literacy', 'mentorship', 'exams', 'books', 'discovery', 'theory', 'practice', 'guidance',
            'discipline', 'college', 'lectures', 'homework', 'projects', 'degrees', 'certificates', 'professors',
            'libraries', 'experiments', 'seminars', 'workshops', 'online learning', 'pedagogy', 'assessment',
            'critical thinking', 'creativity', 'innovation', 'subjects', 'fields', 'specialization',
            'extracurricular', 'tutoring',
            # کلمات فارسی
            'آموزش', 'یادگیری', 'دانش', 'مطالعه', 'مدرسه', 'دانش‌آموز', 'معلم', 'تمرین', 'مهارت',
            'توسعه', 'پژوهش', 'اطلاعات', 'درک', 'تحصیل', 'کلاس', 'درس', 'بورسیه', 'هوش', 'خرد',
            'تدریس', 'برنامه درسی', 'دانشگاه', 'علم', 'سواد', 'راهنمایی', 'امتحان', 'کتاب', 'کشف',
            'نظریه', 'عمل', 'رشته', 'تربیت', 'استاد', 'کالج', 'سخنرانی', 'تکالیف', 'پروژه‌ها',
            'مدرک', 'گواهینامه', 'کتابخانه', 'آزمایش', 'سمینار', 'کارگاه', 'آموزش آنلاین', 'روش تدریس',
            'ارزیابی', 'تفکر انتقادی', 'خلاقیت', 'نوآوری', 'موضوعات', 'حوزه', 'تخصص', 'فعالیت‌های فوق‌برنامه',
            'تدریس خصوصی'
        ],
        '(محیط زیست/environment)': [
            # کلمات انگلیسی
            'environment', 'nature', 'climate', 'sustainability', 'ecology', 'planet', 'resources', 'energy',
            'pollution', 'green', 'earth', 'wildlife', 'conservation', 'weather', 'forests', 'oceans', 'animals',
            'biodiversity', 'recycling', 'natural', 'global', 'warming', 'carbon', 'emissions', 'renewable',
            'habitat', 'soil', 'water', 'air', 'protection', 'ecosystems', 'climate change', 'preservation',
            'deforestation', 'erosion', 'species', 'extinction', 'solar', 'wind', 'hydro', 'geothermal',
            'farming', 'agriculture', 'land', 'urban', 'waste', 'management', 'restoration', 'environmentalism',
            'policy', 'regulation', 'activism', 'research', 'monitoring', 'ozone', 'greenhouse',
            # کلمات فارسی
            'محیط زیست', 'طبیعت', 'آب‌وهوا', 'پایداری', 'اکولوژی', 'سیاره', 'منابع', 'انرژی', 'آلودگی',
            'سبز', 'زمین', 'حیات‌وحش', 'حفاظت', 'هوا', 'جنگل', 'اقیانوس', 'حیوانات', 'تنوع زیستی',
            'بازیافت', 'طبیعی', 'گرمایش جهانی', 'کربن', 'انتشار', 'تجدیدپذیر', 'زیستگاه', 'خاک', 'آب',
            'هوای پاک', 'حفظ', 'اکوسیستم', 'تغییرات اقلیمی', 'نگهداری', 'منابع طبیعی', 'جنگل‌زدایی',
            'فرسایش', 'گونه', 'انقراض', 'خورشیدی', 'بادی', 'آبی', 'زمین‌گرمایی', 'کشاورزی', 'زمین کشاورزی',
            'شهری', 'زباله', 'مدیریت', 'بازسازی', 'محیط‌زیست‌گرایی', 'سیاست', 'مقررات', 'فعالیت زیست‌محیطی',
            'پژوهش', 'نظارت', 'ازون', 'گازهای گلخانه‌ای'
        ],
        '(اقتصاد/economy)': [
            # کلمات انگلیسی
            'economy', 'money', 'finance', 'trade', 'market', 'business', 'investment', 'wealth', 'commerce',
            'industry', 'profit', 'cost', 'budget', 'growth', 'development', 'jobs', 'capital', 'production',
            'consumption', 'financial', 'banking', 'stocks', 'revenue', 'expenses', 'savings', 'taxes',
            'inflation', 'enterprise', 'supply', 'demand', 'currency', 'economic', 'transaction', 'loans',
            'interest', 'credit', 'debt', 'assets', 'liabilities', 'shares', 'dividends', 'markets', 'retail',
            'wholesale', 'exports', 'imports', 'globalization', 'monetary', 'fiscal', 'policy', 'regulation',
            'entrepreneurship', 'startups', 'corporations', 'mergers', 'acquisitions', 'bankruptcy', 'recovery',
            # کلمات فارسی
            'اقتصاد', 'پول', 'مالی', 'تجارت', 'بازار', 'کسب‌وکار', 'سرمایه‌گذاری', 'ثروت', 'بازرگانی',
            'صنعت', 'سود', 'هزینه', 'بودجه', 'رشد', 'توسعه', 'شغل', 'سرمایه', 'تولید', 'مصرف', 'مالیات',
            'بانکداری', 'سهام', 'درآمد', 'خرج', 'پس‌انداز', 'تورم', 'شرکت', 'عرضه', 'تقاضا', 'ارز',
            'اقتصادی', 'تراکنش', 'وام', 'بهره', 'اعتبار', 'بدهی', 'دارایی', 'بدهی‌ها', 'سهم', 'سود سهام',
            'بازارها', 'خرده‌فروشی', 'عمده‌فروشی', 'صادرات', 'واردات', 'جهانی‌سازی', 'پولی', 'مالیاتی',
            'سیاست', 'مقررات', 'کارآفرینی', 'استارتاپ', 'شرکت‌ها', 'ادغام', 'خرید', 'ورشکستگی', 'بهبودی'
        ],
        '(سیاست/politics)': [
            # کلمات انگلیسی
            'politics', 'government', 'policy', 'democracy', 'elections', 'leadership', 'law', 'legislation',
            'power', 'authority', 'citizens', 'rights', 'justice', 'governance', 'administration', 'diplomacy',
            'state', 'nation', 'parliament', 'congress', 'senate', 'parties', 'campaign', 'vote', 'reforms',
            'constitution', 'regulations', 'freedom', 'equality', 'corruption', 'lobbying', 'protests',
            'activism', 'ideology', 'security', 'international', 'treaties', 'alliances', 'conflict', 'peace',
            'bureaucracy', 'public', 'representation', 'opposition', 'debate', 'negotiation', 'sanctions',
            # کلمات فارسی
            'سیاست', 'دولت', 'سیاست‌گذاری', 'دموکراسی', 'انتخابات', 'رهبری', 'قانون', 'قانون‌گذاری',
            'قدرت', 'اقتدار', 'شهروندان', 'حقوق', 'عدالت', 'حکمرانی', 'مدیریت', 'دیپلماسی', 'کشور',
            'ملت', 'پارلمان', 'کنگره', 'سنا', 'احزاب', 'کمپین', 'رای', 'اصلاحات', 'قانون اساسی',
            'مقررات', 'آزادی', 'برابری', 'فساد', 'لابی‌گری', 'اعتراض', 'فعالیت سیاسی', 'ایدئولوژی',
            'امنیت', 'بین‌المللی', 'معاهدات', 'اتحاد', 'مناقشه', 'صلح', 'بوروکراسی', 'عمومی',
            'نمایندگی', 'مخالفت', 'مناظره', 'مذاکره', 'تحریم'
        ],
        '(ورزش/sports)': [
            # کلمات انگلیسی
            'sports', 'exercise', 'fitness', 'competition', 'athletes', 'training', 'games', 'team', 'players',
            'coaching', 'performance', 'strength', 'endurance', 'skills', 'tournament', 'championship', 'score',
            'victory', 'defeat', 'rules', 'equipment', 'stadium', 'fans', 'strategy', 'practice', 'running',
            'swimming', 'football', 'basketball', 'tennis', 'gymnastics', 'cycling', 'boxing', 'wrestling',
            'yoga', 'martial arts', 'olympics', 'records', 'medals', 'referee', 'injury', 'recovery', 'spirit',
            # کلمات فارسی
            'ورزش', 'تمرین', 'تناسب اندام', 'مسابقه', 'ورزشکار', 'آموزش', 'بازی', 'تیم', 'بازیکن',
            'مربیگری', 'عملکرد', 'قدرت', 'استقامت', 'مهارت', 'تورنمنت', 'قهرمانی', 'امتیاز', 'پیروزی',
            'شکست', 'قوانین', 'تجهیزات', 'استادیوم', 'هواداران', 'استراتژی', 'تمرین کردن', 'دویدن',
            'شنا', 'فوتبال', 'بسکتبال', 'تنیس', 'ژیمناستیک', 'دوچرخه‌سواری', 'بوکس', 'کشتی', 'یوگا',
            'هنرهای رزمی', 'المپیک', 'رکورد', 'مدال', 'داور', 'آسیب', 'بهبودی', 'روحیه'
        ],
        '(علم/science)': [
            # کلمات انگلیسی
            'science', 'research', 'experiment', 'discovery', 'knowledge', 'theory', 'hypothesis', 'data',
            'analysis', 'technology', 'physics', 'chemistry', 'biology', 'astronomy', 'mathematics', 'geology',
            'engineering', 'innovation', 'laboratory', 'scientists', 'study', 'evidence', 'facts', 'methods',
            'principles', 'equations', 'models', 'observations', 'breakthrough', 'invention', 'exploration',
            'space', 'energy', 'matter', 'quantum', 'genetics', 'evolution', 'climate', 'environment',
            'particles', 'waves', 'researchers', 'publications', 'peer review', 'funding',
            # کلمات فارسی
            'علم', 'پژوهش', 'آزمایش', 'کشف', 'دانش', 'نظریه', 'فرضیه', 'داده', 'تحلیل', 'فناوری', 'فیزیک',
            'شیمی', 'زیست‌شناسی', 'نجوم', 'ریاضیات', 'زمین‌شناسی', 'مهندسی', 'نوآوری', 'آزمایشگاه',
            'دانشمند', 'مطالعه', 'شواهد', 'حقایق', 'روش', 'اصول', 'معادلات', 'مدل', 'مشاهدات', 'پیشرفت',
            'اختراع', 'اکتشاف', 'فضا', 'انرژی', 'ماده', 'کوانتوم', 'ژنتیک', 'تکامل', 'آب‌وهوا', 'محیط زیست',
            'ذرات', 'موج', 'پژوهشگر', 'انتشارات', 'بررسی همتا', 'بودجه'
        ]
    }
    topic_scores = {}
    for theme, related_words in theme_keywords.items():
        score = sum(keyword_freq.get(word, 0) for word in related_words)
        topic_scores[theme] = score

    sorted_topics = sorted(topic_scores.items(), key=lambda x: x[1], reverse=True)
    topics = [theme for theme, _ in sorted_topics[:num_topics]]

    return {
        'keywords': list(keywords),
        'topics': topics
    }


In [ ]:
text_en = """In today’s rapidly evolving world, technology plays a crucial role in shaping our daily lives.
From smartphones to artificial intelligence, these innovations have transformed the way we communicate, work, and even think.
However, with all these advancements, it’s also essential to preserve and celebrate cultural traditions.
Understanding different cultures not only enriches our personal lives but also strengthens global unity.
Lastly, maintaining a healthy lifestyle, including regular exercise and mindfulness, is vital for personal well-being.
A balanced life helps us stay focused, resilient, and ready to tackle the challenges of modern society."""

text_fa = """در دنیای امروز که به سرعت در حال پیشرفت است، فناوری نقش حیاتی در شکل‌دهی به زندگی روزمره ما دارد.
از گوشی‌های هوشمند گرفته تا هوش مصنوعی، این نوآوری‌ها نحوه ارتباط، کار و حتی تفکر ما را متحول کرده‌اند. با این حال،
با وجود این پیشرفت‌ها، حفظ و گرامی‌داشت سنت‌های فرهنگی نیز ضروری است.
درک فرهنگ‌های مختلف نه تنها زندگی شخصی ما را غنی‌تر می‌کند بلکه وحدت جهانی را نیز تقویت می‌کند. در نهایت،
حفظ سبک زندگی سالم، از جمله ورزش منظم و ذهن‌آگاهی، برای رفاه شخصی بسیار مهم است.
زندگی متعادل به ما کمک می‌کند تا متمرکز، مقاوم و آماده رویارویی با چالش‌های جامعه مدرن باشیم."""

text_en2= """In today’s fast-paced world, technology drives innovation and shapes our lives in countless ways.
Smartphones, artificial intelligence, and cloud computing have revolutionized how we work, communicate, and learn.
At the same time, preserving cultural heritage remains vital, with traditions, music, and literature enriching our communities.
 People strive for a healthy lifestyle, incorporating exercise, meditation, and balanced nutrition to boost physical and mental well-being.
 Education plays a key role, with schools and universities fostering knowledge, creativity, and critical thinking among students. Meanwhile, environmental concerns like climate change,
  pollution, and deforestation demand sustainable solutions, such as renewable energy and recycling. Economically, businesses thrive on trade, investment, and financial strategies, while politics influences governance, democracy, and global alliances.
  Sports bring people together, promoting fitness and teamwork through activities like football, swimming, and yoga. Finally, science pushes boundaries with research in biology, physics, and space exploration, uncovering new possibilities for humanity.
"""

text_fa2 = """در دنیای پرشتاب امروز، فناوری موتور نوآوری است و زندگی ما را به‌شکل‌های بی‌شماری تغییر می‌دهد. گوشی‌های هوشمند، هوش مصنوعی و رایانش ابری نحوه کار،
 ارتباط و یادگیری ما را متحول کرده‌اند. در عین حال، حفظ میراث فرهنگی اهمیت دارد و سنت‌ها، موسیقی و ادبیات جوامع ما را غنی می‌سازند. مردم برای سبک زندگی سالم تلاش می‌کنند و با ورزش، مدیتیشن و تغذیه متعادل به سلامت جسمی و روانی خود کمک می‌کنند. آموزش نقش مهمی دارد و مدارس و دانشگاه‌ها دانش، خلاقیت و تفکر انتقادی را در دانش‌آموزان پرورش می‌دهند.
 در همین حال، مسائل زیست‌محیطی مانند تغییرات اقلیمی، آلودگی و جنگل‌زدایی نیاز به راه‌حل‌های پایدار مثل انرژی تجدیدپذیر و بازیافت دارند. از نظر اقتصادی، کسب‌وکارها با تجارت، سرمایه‌گذاری و استراتژی‌های مالی رشد می‌کنند، در حالی که سیاست بر حکمرانی،
 دموکراسی و اتحادهای جهانی تأثیر می‌گذارد. ورزش مردم را گرد هم می‌آورد و با فعالیت‌هایی مثل فوتبال، شنا و یوگا، تناسب اندام و کار تیمی را ترویج می‌دهد.
 در نهایت، علم با پژوهش در زیست‌شناسی، فیزیک و اکتشاف فضایی، مرزها را جابه‌جا می‌کند و امکانات جدیدی برای بشریت آشکار می‌سازد.
"""

In [ ]:
num_keywords_input = input("تعداد کلمات کلیدی را وارد کنید: ")
num_keywords = int(num_keywords_input) if num_keywords_input.strip() != '' else 5

num_topics_input = input("تعداد موضوعات را وارد کنید: ")
num_topics = int(num_topics_input) if num_topics_input.strip() != '' else 2

result_en = extract_topics_and_keywords(text_en2, num_keywords, num_topics, is_persian=False)
print("keywords:", result_en['keywords'])
print("topics:", result_en['topics'])

result_fa = extract_topics_and_keywords(text_fa2, num_keywords, num_topics, is_persian=True)
print("keywords:", result_fa['keywords'])
print("topics:", result_fa['topics'])

تعداد کلمات کلیدی را وارد کنید: 9
تعداد موضوعات را وارد کنید: 3
keywords: ['activities', 'alliances', 'among', 'artificial', 'balanced', 'biology', 'boost', 'boundaries', 'bring']
topics: ['(سلامتی/health)', '(علم/science)', '(آموزش/education)']
keywords: ['آشکار', 'آلودگی', 'آموز', 'آموزش', 'ابری', 'اتحاد', 'ادبیات', 'ارتباط', 'استراتژی']
topics: ['(سلامتی/health)', '(زندگی/life)', '(آموزش/education)']
